In [3]:
import  pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv('data.csv')

In [5]:
df.drop(columns=['Unnamed: 32'] , inplace=True)

In [6]:
x = df.iloc[: , 2:]
y = df.iloc[: , 1]

In [7]:
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2)

In [8]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [9]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [10]:
class Model(nn.Module):
  def __init__(self , features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(features , 64) ,
        nn.ReLU() ,
        nn.Linear(64 , 32) ,
        nn.ReLU() ,
        nn.Linear(32 , 1),
        nn.Sigmoid()
    )

  def forward(self , features):
    out = self.network(features)
    return out

In [11]:
features = x_train.shape[1]
model = Model(features)

In [12]:
x_train_ten = torch.tensor(np.array(x_train), dtype=torch.float32)
x_test_ten  = torch.tensor(np.array(x_test), dtype=torch.float32)
y_train_ten = torch.tensor(np.array(y_train), dtype=torch.float32).unsqueeze(1)
y_test_ten  = torch.tensor(np.array(y_test), dtype=torch.float32).unsqueeze(1)

In [13]:
rate = 0.1
epochs = 50

In [14]:
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=rate)

In [15]:
for epoch in range(epochs):
  y_pred = model(x_train_ten)

  loss = loss_function(y_pred , y_train_ten)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  print(f"Epoch : {epoch + 1} , Loss : {loss.item()}")

Epoch : 1 , Loss : 0.6876736283302307
Epoch : 2 , Loss : 0.29897111654281616
Epoch : 3 , Loss : 0.9884187579154968
Epoch : 4 , Loss : 0.2695336937904358
Epoch : 5 , Loss : 0.18231429159641266
Epoch : 6 , Loss : 0.17338652908802032
Epoch : 7 , Loss : 0.11795758455991745
Epoch : 8 , Loss : 0.5660873651504517
Epoch : 9 , Loss : 0.17288123071193695
Epoch : 10 , Loss : 0.26653122901916504
Epoch : 11 , Loss : 0.3042166829109192
Epoch : 12 , Loss : 0.30594801902770996
Epoch : 13 , Loss : 0.307279109954834
Epoch : 14 , Loss : 0.3390394151210785
Epoch : 15 , Loss : 0.3353426456451416
Epoch : 16 , Loss : 0.2980585992336273
Epoch : 17 , Loss : 0.2784700393676758
Epoch : 18 , Loss : 0.2839154601097107
Epoch : 19 , Loss : 0.2583392858505249
Epoch : 20 , Loss : 0.25067469477653503
Epoch : 21 , Loss : 0.2479647397994995
Epoch : 22 , Loss : 0.23496674001216888
Epoch : 23 , Loss : 0.24130994081497192
Epoch : 24 , Loss : 0.22663815319538116
Epoch : 25 , Loss : 0.2433825582265854
Epoch : 26 , Loss : 0.22

In [16]:
from torch.utils.data import TensorDataset, DataLoader

test_dataset = TensorDataset(x_test_ten, y_test_ten)
test_loader = DataLoader(test_dataset, batch_size=32)

In [17]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        preds = (outputs > 0.5).float()
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print("Test Accuracy:", accuracy)


Test Accuracy: 0.9824561403508771
